In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import glob
import os
import re

In [3]:
data_dir = '/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc'
misc_dir = '/content/drive/MyDrive/CCRI/ccri_repo/data/misc'

In [4]:
# Read ccri_json
ccri_json = gpd.read_file(os.path.join(data_dir, 'p1_p2_avg_ccri.gpkg'))

# Read UNICEF region mapping
unicef_region = pd.read_csv(os.path.join(misc_dir, 'UNICEF_PROG_REG_GLOBAL.csv'))

In [17]:
# Read and merge hazard exposures
multi_hazards_files = sorted(
    glob.glob(os.path.join(data_dir, '*topics.csv')),
    key=lambda x: int(re.search(r'ge(\d+)_topics', x).group(1)) if re.search(r'ge(\d+)_topics', x) else float('inf')
)
all_hazards = None

for file in multi_hazards_files:
    suffix = file.split('/')[-1].replace('child_pop_exposed_', '').replace('.csv', '')
    df = pd.read_csv(file, usecols=['ISO3', 'pop_exposed']).rename(columns={'pop_exposed': f'{suffix}'})

    # Clean ISO3
    df = df.dropna(subset=['ISO3'])
    df['ISO3'] = df['ISO3'].astype(str).str.strip()
    df = df.drop_duplicates(subset='ISO3')

    if all_hazards is None:
        all_hazards = df
    else:
        all_hazards = all_hazards.merge(df, on='ISO3', how='outer')

# Merge with UNICEF regions (adjust key)
all_hazards = all_hazards.merge(unicef_region[['ISO3Code', 'Region']], left_on='ISO3', right_on='ISO3Code', how='left')
all_hazards.drop(columns=['ISO3Code'], inplace=True)

# Summarize by Region
df_summary_by_region = all_hazards.groupby('Region').sum(numeric_only=True).reset_index()

# Convert to millions and round
df_summary_by_region.iloc[:, 1:] = df_summary_by_region.iloc[:, 1:] / 1_000_000
df_summary_by_region.iloc[:, 1:] = df_summary_by_region.iloc[:, 1:].round(2)

print(df_summary_by_region)


                            Region  ge1_topics  ge2_topics  ge3_topics  \
0            East Asia and Pacific      461.38      347.45      198.42   
1      Eastern and Southern Africa      270.48      199.78       54.55   
2          Europe and Central Asia       69.07       49.61       20.78   
3  Latin America and the Caribbean      164.84      107.46       46.91   
4     Middle East and North Africa      193.07      172.87      124.41   
5                       South Asia      623.42      592.73      374.73   
6          West and Central Africa      301.48      261.55      174.15   

   ge4_topics  ge5_topics  ge6_topics  ge7_topics  ge8_topics  
0       86.43       24.81        3.54        0.10         0.0  
1       14.32        2.51        0.08        0.00         0.0  
2        4.29        0.34        0.01        0.00         0.0  
3       11.56        1.23        0.04        0.00         0.0  
4       47.20        5.20        0.18        0.00         0.0  
5      129.08       15.

In [18]:
# Summarize by Region
df_summary_by_region = all_hazards.groupby('Region').sum(numeric_only=True).reset_index()

# Convert to millions and round
df_summary_by_region.iloc[:, 1:] = df_summary_by_region.iloc[:, 1:] / 1_000_000
df_summary_by_region.iloc[:, 1:] = df_summary_by_region.iloc[:, 1:].round(2)

# Add Total row
total_row = df_summary_by_region.iloc[:, 1:].sum().to_frame().T
total_row.insert(0, 'Region', 'Total')

df_summary_by_region = pd.concat([df_summary_by_region, total_row], ignore_index=True)


print(df_summary_by_region)


                            Region  ge1_topics  ge2_topics  ge3_topics  \
0            East Asia and Pacific      461.38      347.45      198.42   
1      Eastern and Southern Africa      270.48      199.78       54.55   
2          Europe and Central Asia       69.07       49.61       20.78   
3  Latin America and the Caribbean      164.84      107.46       46.91   
4     Middle East and North Africa      193.07      172.87      124.41   
5                       South Asia      623.42      592.73      374.73   
6          West and Central Africa      301.48      261.55      174.15   
7                            Total     2083.74     1731.45      993.95   

   ge4_topics  ge5_topics  ge6_topics  ge7_topics  ge8_topics  
0       86.43       24.81        3.54        0.10         0.0  
1       14.32        2.51        0.08        0.00         0.0  
2        4.29        0.34        0.01        0.00         0.0  
3       11.56        1.23        0.04        0.00         0.0  
4       47.20

In [19]:
df_summary_by_region

,Region,ge1_topics,ge2_topics,ge3_topics,ge4_topics,ge5_topics,ge6_topics,ge7_topics,ge8_topics
0,East Asia and Pacific,461.38,347.45,198.42,86.43,24.81,3.54,0.10,0.0
1,Eastern and Southern Africa,270.48,199.78,54.55,14.32,2.51,0.08,0.00,0.0
2,Europe and Central Asia,69.07,49.61,20.78,4.29,0.34,0.01,0.00,0.0
3,Latin America and the Caribbean,164.84,107.46,46.91,11.56,1.23,0.04,0.00,0.0
4,Middle East and North Africa,193.07,172.87,124.41,47.20,5.20,0.18,0.00,0.0
5,South Asia,623.42,592.73,374.73,129.08,15.52,0.47,0.01,0.0
6,West and Central Africa,301.48,261.55,174.15,36.13,1.82,0.03,0.00,0.0
7,Total,2083.74,1731.45,993.95,329.01,51.43,4.35,0.11,0.0


In [16]:
all_hazards

,ISO3,ge1_topics,ge2_topics,ge3_topics,ge4_topics,ge5_topics,ge6_topics,ge7_topics,ge8_topics,ge9_topics,ge10_topics,Region
0,ABW,2.256094e+04,2.166847e+04,1.896957e+04,3.559946e+03,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,NaN
1,AFG,2.094996e+07,2.092073e+07,2.052212e+07,1.828499e+07,1.088582e+07,2.040468e+06,163137.988984,3524.348965,0.0,0.0,South Asia
2,AGO,1.897740e+07,1.892438e+07,1.781823e+07,1.180709e+07,2.712698e+06,2.828817e+05,8906.990243,0.000000,0.0,0.0,Eastern and Southern Africa
3,AIA,3.107598e+03,3.107598e+03,3.107598e+03,1.620139e+03,1.495599e+01,0.000000e+00,0.000000,0.000000,0.0,0.0,Latin America and the Caribbean
4,ALA,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
257,xSI,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,NaN
258,xSK,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,NaN
259,xSR,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,NaN
260,xUK,1.649855e+03,1.390915e+03,5.820051e+02,2.485924e+02,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,NaN


In [20]:
all_hazards.to_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/misc/MHC_climate_stat.csv', index=False)